In [1]:
from openai import OpenAI
import os
from dotenv import load_dotenv

load_dotenv()

api_key = os.getenv("GENERAL_API_KEY")
base_url = os.getenv("GENERAL_BASE_URL")

client = OpenAI(api_key=api_key,base_url=base_url)

In [3]:
model_name='gpt-4o'

In [23]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_flight_number",
            "description": "根据始发地、目的地和日期，查询对应日期的航班号",
            "parameters": {
                "type": "object",
                "properties": {
                    "departure": {
                        "description": "出发地",
                        "type": "string"
                    },
                    "destination": {
                        "description": "目的地",
                        "type": "string"
                    },
                    "date": {
                        "description": "日期",
                        "type": "string",
                    }
                },
                "required": [ "departure", "destination", "date" ]
            },
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_ticket_price",
            "description": "根据日期和航班号，查询对应航班的票价",
            "parameters": {
                "type": "object",
                "properties": {
                    "flight_number": {
                        "description": "航班号",
                        "type": "string"
                    },
                    "date": {
                        "description": "日期",
                        "type": "string",
                    }
                },
                "required": [ "flight_number", "date"]
            },
        }
    },
]

In [11]:
messages = []
messages.append({"role": "user", "content": "帮我查询从2024年1月20日，从北京出发前往上海的航班"})
response = client.chat.completions.create(
    model=model_name,  # 填写需要调用的模型名称
    messages=messages,
    tools=tools,
)
print(response.choices[0].message)
messages.append(response.choices[0].message.model_dump())

ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_OCL8cnJLKMzDCnITNnzvnk4M', function=Function(arguments='{"date":"2024-01-20","departure":"北京","destination":"上海"}', name='get_flight_number'), type='function')])


In [21]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_ticket_price",
            "description": "根据日期和航班号，查询对应航班的票价",
            "parameters": {
                "type": "object",
                "properties": {
                    "flight_number": {
                        "description": "航班号",
                        "type": "string"
                    },
                    "date": {
                        "description": "日期",
                        "type": "string",
                    }
                },
                "required": [ "flight_number", "date"]
            },
        }
    },
]

gpt-4o模型收到tools内多函数的扰乱问题，导致无法正确调用对应的function，而会一直不停询问查询票价需要相应的出发地和目的地。

glm-4-flash可以在这一步调用正常函数

In [24]:
messages = []
messages.append({"role": "system", "content": "不要假设或猜测传入函数的参数值。如果用户的描述不明确，请要求用户提供必要信息"})
messages.append({"role": "user", "content": "查询2024年1月20日1234航班的票价"})
# messages.append({"role": "user", "content": "查询2024年1月20日1234航班"}) #GLM能查询到，GPT-4o不行，其实应该是glm有问题
response = client.chat.completions.create(
    model=model_name,  # 填写需要调用的模型名称
    messages=messages,
    tools=tools,
)
print(response.choices[0].message)
messages.append(response.choices[0].message.model_dump())

ChatCompletionMessage(content='请提供航班的出发地和目的地，以便我能够更准确地查询1234航班的票价。', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None)


In [25]:
def get_flight_number(date:str , departure:str , destination:str):
    flight_number = {
        "北京":{
            "上海" : "1234",
            "广州" : "8321",
        },
        "上海":{
            "北京" : "1233",
            "广州" : "8123",
        }
    }
    return { "flight_number":flight_number[departure][destination] }
def get_ticket_price(date:str , flight_number:str):
    return {"ticket_price": "1000"}

In [26]:
import json


def parse_function_call(model_response,messages):
    # 处理函数调用结果，根据模型返回参数，调用对应的函数。
    # 调用函数返回结果后构造tool message，再次调用模型，将函数结果输入模型
    # 模型会将函数调用结果以自然语言格式返回给用户。
    if model_response.choices[0].message.tool_calls:
        tool_call = model_response.choices[0].message.tool_calls[0]
        args = tool_call.function.arguments
        function_result = {}
        if tool_call.function.name == "get_flight_number":
            function_result = get_flight_number(**json.loads(args))
        if tool_call.function.name == "get_ticket_price":
            function_result = get_ticket_price(**json.loads(args))
        messages.append({
            "role": "tool",
            "content": f"{json.dumps(function_result)}",
            "tool_call_id":tool_call.id
        })
        response = client.chat.completions.create(
            model=model_name,  # 填写需要调用的模型名称
            messages=messages,
            tools=tools,
        )
        print(response.choices[0].message)
        messages.append(response.choices[0].message.model_dump())

In [33]:
# 清空对话
messages = []
 
messages.append({"role": "system", "content": "不要假设或猜测传入函数的参数值。如果用户的描述不明确，请要求用户提供必要信息"})
messages.append({"role": "user", "content": "帮我查询1月23日，北京到广州的航班"})
 
response = client.chat.completions.create(
    model=model_name, # 填写需要调用的模型名称
    messages=messages,
    tools=tools,
)
print(response.choices[0].message)
messages.append(response.choices[0].message.model_dump())
 
parse_function_call(response,messages)

ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_VP5VZLXcSA3sZkwnEZwir5rn', function=Function(arguments='{"date":"2024-01-23","departure":"北京","destination":"广州"}', name='get_flight_number'), type='function')])
ChatCompletionMessage(content='1月23日从北京飞往广州的航班号是8321。您需要查询票价吗？', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None)


In [34]:
messages

[{'role': 'system', 'content': '不要假设或猜测传入函数的参数值。如果用户的描述不明确，请要求用户提供必要信息'},
 {'role': 'user', 'content': '帮我查询1月23日，北京到广州的航班'},
 {'content': None,
  'refusal': None,
  'role': 'assistant',
  'audio': None,
  'function_call': None,
  'tool_calls': [{'id': 'call_VP5VZLXcSA3sZkwnEZwir5rn',
    'function': {'arguments': '{"date":"2024-01-23","departure":"北京","destination":"广州"}',
     'name': 'get_flight_number'},
    'type': 'function'}]},
 {'role': 'tool',
  'content': '{"flight_number": "8321"}',
  'tool_call_id': 'call_VP5VZLXcSA3sZkwnEZwir5rn'},
 {'content': '1月23日从北京飞往广州的航班号是8321。您需要查询票价吗？',
  'refusal': None,
  'role': 'assistant',
  'audio': None,
  'function_call': None,
  'tool_calls': None}]

gpt-4o在这里可以根据上下文获取航班票价的完整调用function参数信息

glm-4-flash不行

In [35]:
messages.append({"role": "user", "content": "这趟航班的价格是多少？"})
# messages.append({"role": "user", "content": "这趟航班的票价是多少？"})
response = client.chat.completions.create(
    model=model_name,  # 填写需要调用的模型名称
    messages=messages,
    tools=tools,
)
print(response.choices[0].message)
messages.append(response.choices[0].message.model_dump())
 
parse_function_call(response,messages)

ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_rTiADgkjslQZJCcAc8tz7kqF', function=Function(arguments='{"date":"2024-01-23","flight_number":"8321"}', name='get_ticket_price'), type='function')])
ChatCompletionMessage(content='这趟航班（航班号8321）的票价是1000元。', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None)
